In [1]:
import pandas as pd
from datetime import datetime
from functions_load_and_transform import thisWeek_DF, lastWeek_DF, thisWeek, lastWeek, team_list, scoringDF, player_list

In [ ]:
selected_week = 3

playerDF = pd.DataFrame(data={"Spieler":player_list}) # sums up 
playerDF["Gesamtpunkte"] = scoringDF.loc[scoringDF["Week"]<=selected_week, player_list].sum().to_list()
# playerDF["Last Week"] = scoringDF.loc[scoringDF["Week"]<=selected_week-1, player_list].sum().to_list()
playerDF["Wöchentliche Punkte"] = scoringDF.loc[scoringDF["Week"]==selected_week, player_list].sum().to_list()